<a href="https://colab.research.google.com/github/supervik/binance-trading-performance/blob/master/Binance_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binance trading performance evaluation

## User settings

In [ ]:
# @markdown *API keys. Use read-only API keys for more security:*
API_key = 'Rk39fQkdqoO3zYeud4b6dBGjMzMPkxVCVNUODIUMiBe8PaTTiDJk8v0YLy' #@param {type:"string"}
API_secret = 'Gja2onh601kxZlaClp01mazkdd4b6dBGjMzM6roP29skq7EQDK4b5g0o' #@param {type:"string"}

# @markdown *Market. Use dash '-' between base and quote assets.*
market = 'XZC-BTC' #@param {type:"string"}
 
# @markdown *Start and end date (UTC). Note, that **end_date** is used for getting final prices. P&L greatly depends on this date!*
start_date = "2020-07-07" #@param {type:"date"}
end_date = '2020-07-13' #@param {type:"date"}

## Binance API

In [ ]:
#@title
# Credits to @Bablofil https://github.com/Bablofil/binance-api
import time
import json
import urllib
import hmac, hashlib
import requests

from urllib.parse import urlparse, urlencode
from urllib.request import Request, urlopen

class Binance():
    
    methods = {
            #  Public methods     
            'ping':             {'url': 'ping', 'method': 'GET', 'private': False},                     
            'time':             {'url': 'time', 'method': 'GET', 'private': False},         
            'exchangeInfo':     {'url': 'exchangeInfo', 'method': 'GET', 'private': False},
            'depth':            {'url': 'depth', 'method': 'GET', 'private': False},
            'trades':           {'url': 'trades', 'method': 'GET', 'private': False},  
            'historicalTrades': {'url': 'historicalTrades', 'method': 'GET', 'private': False},  
            'aggTrades':        {'url': 'aggTrades', 'method': 'GET', 'private': False},
            'klines':           {'url': 'klines', 'method': 'GET', 'private': False},   
            'avgPrice':         {'url': 'avgPrice', 'method': 'GET', 'private': False}, 
            'ticker24hr':       {'url': 'ticker/24hr', 'method': 'GET', 'private': False},
            'tickerPrice':      {'url': 'ticker/price', 'method': 'GET', 'private': False},   
            'tickerBookTicker': {'url': 'ticker/bookTicker', 'method': 'GET', 'private': False}, 
            #  Private methods   
            'createOrder':      {'url': 'order', 'method': 'POST', 'private': True},                        
            'testOrder':        {'url': 'test', 'method': 'POST', 'private': True},              
            'orderInfo':        {'url': 'order', 'method': 'GET', 'private': True},
            'cancelOrder':      {'url': 'order', 'method': 'DELETE', 'private': True},
            'openOrders':       {'url': 'openOrders', 'method': 'GET', 'private': True}, 
            'allOrders':        {'url': 'allOrders', 'method': 'GET', 'private': True},     
            'account':          {'url': 'account', 'method': 'GET', 'private': True}, 
            'myTrades':         {'url': 'myTrades', 'method': 'GET', 'private': True}, 
   }
    
    def __init__(self, API_KEY, API_SECRET):
        self.API_KEY = API_KEY
        self.API_SECRET = bytearray(API_SECRET, encoding='utf-8')
        self.shift_seconds = 0

    def __getattr__(self, name):
        def wrapper(*args, **kwargs):
            kwargs.update(command=name)
            return self.call_api(**kwargs)
        return wrapper

    def set_shift_seconds(self, seconds):
        self.shift_seconds = seconds
        
    def call_api(self, **kwargs):

        command = kwargs.pop('command')
        api_url = 'https://api.binance.com/api/v3/' + self.methods[command]['url']

        payload = kwargs
        headers = {}
        
        payload_str = urllib.parse.urlencode(payload)
        if self.methods[command]['private']:
            payload.update({'timestamp': int(time.time() + self.shift_seconds - 1) * 1000})
            payload_str = urllib.parse.urlencode(payload).encode('utf-8')
            sign = hmac.new(
                key=self.API_SECRET,
                msg=payload_str,
                digestmod=hashlib.sha256
            ).hexdigest()

            payload_str = payload_str.decode("utf-8") + "&signature="+str(sign) 
            headers = {"X-MBX-APIKEY": self.API_KEY}

        if self.methods[command]['method'] == 'GET':
            api_url += '?' + payload_str

        # print(api_url, payload_str, self.methods[command])
        response = requests.request(
            method=self.methods[command]['method'], 
            url=api_url, 
            data="" if self.methods[command]['method'] == 'GET' else payload_str,
            headers=headers)

        if 'code' in response.text:
            print(response.text)
            response = response.json()
            return response['error']
        return response.json()



## Calculations

In [ ]:
#@title
import sys
import time
import calendar
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)

try:
    asset_base = market.split("-")[0]
    asset_quote = market.split("-")[1]
except IndexError:
    raise Exception(f"!!! Warning: Use dash '-' to split base and quote assets for {market}!")

symbol = asset_base + asset_quote

# Connect to Binance
binance = Binance(API_key, API_secret)
try:
    trades = binance.myTrades(symbol = symbol)
    print(f"INFO: Exchange is successfully connected.")
except:
    raise Exception(f"!!! Warning: Can't get orders for {symbol}! Read message above.")    

# Create DataFrame 
df = pd.DataFrame(trades, columns = ['time', 'symbol', 'isBuyer', 'price', 'qty', 'quoteQty', 'commission', 'commissionAsset'])
qty_base = 'qty_' + asset_base
qty_quote = 'qty_' + asset_quote
df.columns = ['time', 'symbol', 'side', 'price', qty_base, qty_quote, 'fee', 'fee_coin']
df.side = df.side.replace([True, False], [1, -1])
df = df.astype({'price': 'float', qty_base: 'float', qty_quote: 'float', 'fee': 'float'})

# Start from time
time_format = '%Y-%m-%d'
start_date_ms = int(calendar.timegm(time.strptime(start_date, time_format)) * 1000)
end_date_ms = int((calendar.timegm(time.strptime(end_date, time_format))) * 1000)
df = df[(df.time >= start_date_ms) & (df.time <= end_date_ms + 86_400_000)]
df.time = pd.to_datetime(df.time, unit='ms')

# Find time for getting market prices
time_now = time.gmtime(time.time())
day_now_ms = calendar.timegm((time_now.tm_year, time_now.tm_mon, time_now.tm_mday, 0, 0, 0, 0, 0, 0)) * 1000
prices_time = min(day_now_ms, end_date_ms)

# Get symbol price
try:
    symbol_price  = float(binance.klines(symbol = symbol, interval = '1m', startTime = prices_time, limit = 1)[0][1])
except:
    print(f"Something wrong with request of {symbol} price. Please try again.")

# Get quote-USD price
if asset_quote == 'USDC' or asset_quote == 'USDT' or asset_quote == 'BUSD':
    usd_price = 1
else:
    try:
        usd_price  = float(binance.klines(symbol = asset_quote + 'USDT', interval = '1m', startTime = prices_time, limit = 1)[0][1])
    except:
        print(f"Something wrong with the request of {asset_quote}USDT price. Please try again.")

# Get BNB-quote price
if asset_quote == 'BNB':
    bnb_price = 1
else:
    try:
        bnb_price = float(binance.klines(symbol = 'BNB' + asset_quote, interval = '1m', startTime = prices_time, limit = 1)[0][1])
    except:
        print(f"Something wrong with the request of BNB{asset_quote} price. Please try again.")

# Summary
days = int((prices_time - start_date_ms)/(1000 * 86400)) + 1
average_buy = df[df.side == 1][qty_quote].sum()/df[df.side == 1][qty_base].sum()
average_sell = df[df.side == -1][qty_quote].sum()/df[df.side == -1][qty_base].sum()
total_volume = df[qty_quote].sum()

# Delta
delta_base = (df[qty_base] * df.side).sum()
delta_quote = - (df[qty_quote] * df.side).sum()

# Fees
fee_bnb = df[df.fee_coin == 'BNB'].fee.sum()
fee_base = df[df.fee_coin == asset_base].fee.sum()
fee_quote = df[df.fee_coin == asset_quote].fee.sum()

# Totals
total_quote = (delta_base - fee_base) * symbol_price + (delta_quote - fee_quote) - fee_bnb * bnb_price
total_usd = total_quote * usd_price
prices_time_utc = time.strftime(time_format, time.gmtime(prices_time/1000))

## Trades

In [ ]:
#@title
if df.empty:
  print(f"No trades found for {symbol} from {start_date} till {end_date}")
else: 
  print(f"\nTrades gathered for {symbol}:")
  df.side = df.side.replace([1, -1], ['BUY', 'SELL'])
  df

## Summary

In [ ]:
#@title
print(f"Summary for {symbol} for period [{start_date} - {end_date}]:")
print(f"   Days: {days}")
print(f"   Trades executed: {df.time.count()}")
print(f"   Total volume traded ({asset_quote}): {round(total_volume, 8)}")
print(f"   Average buy price: {round(average_buy, 8)}")
print(f"   Average sell price: {round(average_sell, 8)}")
print(f"\nTrading delta:")
print(f"   Delta {asset_base}: {round(delta_base, 8)}")
print(f"   Delta {asset_quote}: {round(delta_quote, 8)}")
print(f"\nFees:")
print(f"   Fees BNB: {round(fee_bnb, 8)}")
print(f"   Fees {asset_base}: {round(fee_base, 8)}")
print(f"   Fees {asset_quote}: {round(fee_quote, 8)}")
print(f"\nPrices at the end of the period [{prices_time_utc}]:")
print(f"   Price {symbol}: {symbol_price}")
print(f"   Price {asset_quote}USDT: {usd_price}")
print(f"   Price BNB{asset_quote}: {bnb_price}")
print(f"\nTotal profit:")
print(f"   Total profit ({asset_quote}): {round(total_quote, 8)}")
print(f"   Total profit (USDT): {round(total_usd, 8)}")